# Preprocessing CyGNSS data

Data is downloaded from NASA EarthCloud as described in the `APIs` notebook. For the expected format for CyGNSSnet, additional preprocessing steps are necessary.

In [ ]:
import os
import sys
sys.path.append('/work/ka1176/caroline/gitlab/2020-03-gfz-remote-sensing/')

In [ ]:
from gfz_202003.preprocessing import preprocess as prep

In [ ]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import seaborn as sns

import datetime
import xarray as xr

In [ ]:
import cdsapi

## Download raw CyGNSS data

The CyGNSSnet preprocessing routine expects the raw data files ordered as 

> `$raw_data_dir/<year>/<day-of-year>/cyg*.nc`

Data is always downloaded for one full day for all spacecraft, generating 8 `netcdf` files per day of observations. Below is a routine to specify a date range, followed by downloading the corresponding data and storing it in the appropriate subfolders.

In [ ]:
raw_data_root = '/work/ka1176/shared_data/2022-cygnss-deployment/raw_data/'
dev_data_root = '/work/ka1176/shared_data/2022-cygnss-deployment/dev_data/'

Select a test day and prepared the input parameters for the provided download script

In [ ]:
year  = 2021
month = 6
day   = 17

Downloaded target directory in the expected format `year/day-of-year`

In [ ]:
raw_data_sub = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%Y/%j")

raw_data_dir = os.path.join(raw_data_root, raw_data_sub)

print(raw_data_dir)

Start and end date of download range in the required format. The end date is midnight the next day, this way only the requested day's data is downloaded.

In [ ]:
start_date = datetime.datetime(year, month, day).strftime("%Y-%m-%dT%H:%M:%SZ")
end_date   = datetime.datetime(year, month, day + 1).strftime("%Y-%m-%dT%H:%M:%SZ")

print(f'--start-date {start_date}')
print(f'--end-date   {end_date}')

In [ ]:
%env PYTHONPATH=/work/ka1176/caroline/gitlab/data-subscriber
!python /work/ka1176/caroline/gitlab/data-subscriber/subscriber/podaac_data_downloader.py  -c CYGNSS_L1_V3.1 -d $raw_data_dir --start-date $start_date --end-date $end_date

## Download raw ERA5 data

The preprocessing pipeline requires the ERA5 windspeed labels. Download the raw ERA5 data for the same timespan.

In [ ]:
era5_data = os.path.join(raw_data_dir, 'ERA5_windspeed.nc')

In [ ]:
cds = cdsapi.Client()

In [ ]:
cds.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind',
        ],
        'year': year,
        'month': month,
        'day': day,
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00'
        ],
        'area': [
            40, -180, -40, 180,
        ],
    },
    era5_data)

In [ ]:
era5_ds = xr.open_dataset(era5_data)
era5_ds

## Annotate raw CyGNSS data with windspeed labels

We need to create the data variables `ERA5_u10` and `ERA5_v10` and attach them to the CyGNSS raw data.

In [ ]:
os.listdir(raw_data_dir)

In [ ]:
cygnss_file = os.path.join(raw_data_dir, 'cyg06.ddmi.s20210617-000000-e20210617-235959.l1.power-brcs.a31.d32.nc')

# necessary because lazy loading prohibits overwriting the netcdf files at the end of this section
with xr.open_dataset(cygnss_file) as data:
    cygnss_ds = data.load()
    
#cygnss_ds.close() # close I/O stream
cygnss_ds

Check units for spacetime coordinates
* Longitude
  * ERA5:   -180 ... 0 ... +180
  * CyGNSS: 0 ... 180 ... 360
* Latitude
  * ERA5 & CyGNSS: -40 ... 0 ... +40
* Timestamp


--> Need to shift the ERA5 longitude coordinate by 180

In [ ]:
era5_ds = era5_ds.assign_coords(longitude=era5_ds.coords['longitude'] + 180)
era5_ds

In [ ]:
sample_ix = 4821
ddm_ix    = 2

sample_ds = cygnss_ds.sel(sample=sample_ix, ddm=ddm_ix)
sample_ds

sample_sp_lat = sample_ds.sp_lat.values
print(sample_sp_lat)

sample_sp_lon = sample_ds.sp_lon.values
print(sample_sp_lon)

sample_ddm_timestamp_utc = sample_ds.ddm_timestamp_utc.values
print(sample_ddm_timestamp_utc)

In [ ]:
interp_ds = era5_ds.interp(longitude=cygnss_ds.sp_lon, latitude=cygnss_ds.sp_lat, time=cygnss_ds.ddm_timestamp_utc)
interp_ds

In [ ]:
cygnss_ds['ERA5_u10'] = interp_ds['u10']
cygnss_ds['ERA5_v10'] = interp_ds['v10']

tmp_attrs = cygnss_ds['ERA5_u10'].attrs
tmp_attrs['long_name'] = cygnss_ds['ERA5_u10'].long_name + ' (interpolated)'
cygnss_ds['ERA5_u10'].attrs = tmp_attrs

tmp_attrs = cygnss_ds['ERA5_v10'].attrs
tmp_attrs['long_name'] = cygnss_ds['ERA5_v10'].long_name + ' (interpolated)'
cygnss_ds['ERA5_v10'].attrs = tmp_attrs

In [ ]:
cygnss_ds = cygnss_ds.drop_vars(['longitude', 'latitude', 'time'])

TODO: drop the variables that are not used later on and apply the quality filter here

Overwrite the dataset on disk

In [ ]:
cygnss_ds.to_netcdf(cygnss_file)

## Check raw data

In [ ]:
from importlib import reload
reload(prep)
raw_ds = prep.open_mfdataset(os.path.join(raw_data_dir, cygnss_file))

raw_ds

In [ ]:
filtered_ds = prep.apply_quality_filter(raw_ds, is_ml_ops=True)
filtered_ds

In [ ]:
os.listdir('/work/ka1176/shared_data/2020-03/raw_data/2021/014/')

In [ ]:
old_raw_ds = xr.open_mfdataset('/work/ka1176/shared_data/2020-03/raw_data/2022/168/cyg06.ddmi.s20210114-000000-e20210114-235959.l1.power-brcs.a30.d31.nc')
old_raw_ds

In [ ]:
bu = raw_ds['ddm_brcs_uncert']
qf = raw_ds['quality_flags']
st = raw_ds['nst_att_status']
fom = raw_ds['prn_fig_of_merit']
les = raw_ds['ddm_les']
rxg = raw_ds['sp_rx_gain']
nsca = raw_ds['nbrcs_scatter_area']
lsca = raw_ds['les_scatter_area']
lat = raw_ds['sp_lat']
lon = raw_ds['sp_lon']
ws = raw_ds['windspeed']

TODO issue with the quality flag

In [ ]:
quality = (bu<1) & (qf == 4) & (st == 0) & (fom > 3) & (rxg > 0) & (les >= 0)

In [ ]:
np.sum((bu<1) & (st==0)).compute()

## Created processed data

In [ ]:
raw_files = prep.open_mfdataset(os.path.join(raw_data_dir, '*.nc'), channels=[0,1,2,3])

In [ ]:
raw_ds = xr.open_mfdataset(os.path.join(raw_data_dir, '*.nc'), engine="netcdf4")

raw_ds

In [ ]:
raw_ds = xr.open_mfdataset([os.path.join(raw_data_dir, dd) for dd in os.listdir(raw_data_dir) if dd.endswith('.nc')])

## Check the new CyGNSS data v3.1

In [ ]:
cygnss_ds

In [ ]:
qf = cygnss_ds.quality_flags[:100].values

In [ ]:
qf == 4

In [ ]:
bu = cygnss_ds.ddm_brcs_uncert[:100].values

In [ ]:
cygnss_ds.ddm_brcs_uncert

In [ ]:
brcs = cygnss_ds.brcs[1].values

In [ ]:
np.sum(brcs, axis=(1,2))

In [ ]:
cygnss_ds.quality_flags[1].values

In [ ]:
plt.imshow(raw_ds.brcs[0].values)
plt.show()
print(raw_ds.windspeed[0].values)